Use the postal codes of Canada to get postal code, Borough and neighbourhood

In [1]:
import pandas as pd

import requests
import scrapy

from bs4 import BeautifulSoup

req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

soup = BeautifulSoup(req.content,'lxml')

table = soup.find_all('table')[0]

df = pd.read_html(str(table))

neighborhood=pd.DataFrame(df[0])

In [2]:
neighborhood = neighborhood[neighborhood['Borough']!='Not assigned']

In [3]:
temp_df=neighborhood.groupby('Postal code')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [4]:
neighborhood_merge = pd.merge(neighborhood, temp_df, on='Postal code')

In [5]:
neighborhood_merge.drop_duplicates(inplace=True)

In [6]:
neighborhood_merge.drop(['Neighborhood'],axis=1,inplace=True)
neighborhood_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [7]:
neighborhood_merge.shape

(103, 3)

In [8]:
neighborhood_merge

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


Find the geographical data and merge it with the neighborhood data to get the longitude and latitude of each area

In [9]:
geo_df=pd.read_csv('Geospatial_Coordinates.csv')

In [10]:
neighborhood_merge.rename(columns={'Postal code':'Postal Code'},inplace=True)
geo_merged = pd.merge(geo_df, neighborhood_merge, on='Postal Code')

In [11]:
geo_merged.rename(columns={'Postal Code':'PostalCode'},inplace=True)

In [12]:
toronto_data=geo_merged[geo_merged['Borough'].str.contains("Toronto")]
toronto_data.head()

,PostalCode,Latitude,Longitude,Borough,Neighborhood
37,M4E,43.676357,-79.293031,East Toronto,The Beaches
41,M4K,43.679557,-79.352188,East Toronto,The Danforth West / Riverdale
42,M4L,43.668999,-79.315572,East Toronto,India Bazaar / The Beaches West
43,M4M,43.659526,-79.340923,East Toronto,Studio District
44,M4N,43.728020,-79.388790,Central Toronto,Lawrence Park


Use foursquare api to get list of venues nearby

In [13]:
from geopy.geocoders import Nominatim 

In [14]:
CLIENT_ID = 'OTGQCNYFCE3Z4J53TBTZB3XF2DZ1UHMTGL5FUDH1PKWFBHKZ' # your Foursquare ID
CLIENT_SECRET = 'VLB1JAFUAMIN1QQGWY4MBK45QEAJERFKHGX3T3OQ5MSOBISJ' # your Foursquare Secret
VERSION = '20180604'

In [15]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West / Riverdale
India Bazaar / The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park / Summerhill East
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
Rosedale
St. James Town / Cabbagetown
Church and Wellesley
Regent Park / Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond / Adelaide / King
Harbourfront East / Union Station / Toronto Islands
Toronto Dominion Centre / Design Exchange
Commerce Court / Victoria Hotel
Roselawn
Forest Hill North & West
The Annex / North Midtown / Yorkville
University of Toronto / Harbord
Kensington Market / Chinatown / Grange Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
Stn A PO Boxes
First Canadian Place / Underground city
Christie
Dufferin / Dovercourt Village
Little Portugal / Trinity
Brockton / Parkdale Village / Exhibition Place
High Park / 

In [17]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
Brockton / Parkdale Village / Exhibition Place,22,22,22,22,22,22
Business reply mail Processing CentrE,17,17,17,17,17,17
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,16,16,16,16,16,16
Central Bay Street,67,67,67,67,67,67
Christie,17,17,17,17,17,17
Church and Wellesley,71,71,71,71,71,71
Commerce Court / Victoria Hotel,100,100,100,100,100,100
Davisville,36,36,36,36,36,36


In [18]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Danforth West / Riverdale,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [19]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )
toronto_onehot.shape

(1612, 230)

In [20]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,0.0,0.017241,...,0.0,0.0,0.0,0.017241,0.0,0.0,0.0,0.0,0.0,0.000000
1,Brockton / Parkdale Village / Exhibition Place,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
2,Business reply mail Processing CentrE,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.058824
3,CN Tower / King and Spadina / Railway Lands / ...,0.0625,0.0625,0.125,0.1875,0.125,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
4,Central Bay Street,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.014925,0.0,0.0,0.0,0.0,0.0,0.014925


In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
import numpy as np

In [23]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café
1,Brockton / Parkdale Village / Exhibition Place,Café,Coffee Shop,Breakfast Spot,Performing Arts Venue,Bakery
2,Business reply mail Processing CentrE,Yoga Studio,Fast Food Restaurant,Comic Shop,Park,Pizza Place
3,CN Tower / King and Spadina / Railway Lands / ...,Airport Service,Airport Lounge,Airport Terminal,Airport,Boat or Ferry
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Salad Place


In [24]:
from sklearn.cluster import KMeans

In [25]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [26]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Latitude,Longitude,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
37,M4E,43.676357,-79.293031,East Toronto,The Beaches,0,Health Food Store,Trail,Pub,Yoga Studio,Dessert Shop
41,M4K,43.679557,-79.352188,East Toronto,The Danforth West / Riverdale,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Ice Cream Shop
42,M4L,43.668999,-79.315572,East Toronto,India Bazaar / The Beaches West,1,Pizza Place,Fast Food Restaurant,Park,Steakhouse,Sushi Restaurant
43,M4M,43.659526,-79.340923,East Toronto,Studio District,1,Café,Coffee Shop,American Restaurant,Bakery,Brewery
44,M4N,43.728020,-79.388790,Central Toronto,Lawrence Park,2,Park,Bus Line,Swim School,Diner,Event Space


In [27]:
toronto_merged

,PostalCode,Latitude,Longitude,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
37,M4E,43.676357,-79.293031,East Toronto,The Beaches,0,Health Food Store,Trail,Pub,Yoga Studio,Dessert Shop
41,M4K,43.679557,-79.352188,East Toronto,The Danforth West / Riverdale,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Ice Cream Shop
42,M4L,43.668999,-79.315572,East Toronto,India Bazaar / The Beaches West,1,Pizza Place,Fast Food Restaurant,Park,Steakhouse,Sushi Restaurant
43,M4M,43.659526,-79.340923,East Toronto,Studio District,1,Café,Coffee Shop,American Restaurant,Bakery,Brewery
44,M4N,43.728020,-79.388790,Central Toronto,Lawrence Park,2,Park,Bus Line,Swim School,Diner,Event Space
45,M4P,43.712751,-79.390197,Central Toronto,Davisville North,1,Gym,Hotel,Department Store,Sandwich Place,Breakfast Spot
46,M4R,43.715383,-79.405678,Central Toronto,North Toronto West,1,Clothing Store,Coffee Shop,Yoga Studio,Sporting Goods Shop,Café
47,M4S,43.704324,-79.388790,Central Toronto,Davisville,1,Pizza Place,Sandwich Place,Dessert Shop,Coffee Shop,Sushi Restaurant
48,M4T,43.689574,-79.383160,Central Toronto,Moore Park / Summerhill East,4,Playground,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space
49,M4V,43.686412,-79.400049,Central Toronto,Summerhill West / Rathnelly / South Hill / For...,1,Coffee Shop,Pub,Pizza Place,Bank,Sushi Restaurant


In [28]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café
1,1,Brockton / Parkdale Village / Exhibition Place,Café,Coffee Shop,Breakfast Spot,Performing Arts Venue,Bakery
2,1,Business reply mail Processing CentrE,Yoga Studio,Fast Food Restaurant,Comic Shop,Park,Pizza Place
3,1,CN Tower / King and Spadina / Railway Lands / ...,Airport Service,Airport Lounge,Airport Terminal,Airport,Boat or Ferry
4,1,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Salad Place


In [29]:
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café
1,1,Brockton / Parkdale Village / Exhibition Place,Café,Coffee Shop,Breakfast Spot,Performing Arts Venue,Bakery
2,1,Business reply mail Processing CentrE,Yoga Studio,Fast Food Restaurant,Comic Shop,Park,Pizza Place
3,1,CN Tower / King and Spadina / Railway Lands / ...,Airport Service,Airport Lounge,Airport Terminal,Airport,Boat or Ferry
4,1,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Salad Place
5,1,Christie,Grocery Store,Café,Park,Restaurant,Italian Restaurant
6,1,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Yoga Studio
7,1,Commerce Court / Victoria Hotel,Coffee Shop,Café,Restaurant,Hotel,Gym
8,1,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Coffee Shop,Sushi Restaurant
9,1,Davisville North,Gym,Hotel,Department Store,Sandwich Place,Breakfast Spot


In [30]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.6534817, -79.3839347.


In [31]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

In [32]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        colour='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [33]:
toronto_food_venues = toronto_venues[toronto_venues['Venue Category'].str.contains("Food")]

In [34]:
toronto_food_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
49,India Bazaar / The Beaches West,43.668999,-79.315572,The Burger's Priest,43.666731,-79.315556,Fast Food Restaurant
62,India Bazaar / The Beaches West,43.668999,-79.315572,KFC,43.666624,-79.315916,Fast Food Restaurant
80,Studio District,43.659526,-79.340923,eastside social,43.661289,-79.339155,Comfort Food Restaurant
109,Davisville North,43.712751,-79.390197,Summerhill Market North,43.715499,-79.392881,Food & Drink Shop
126,North Toronto West,43.715383,-79.405678,A&W,43.715149,-79.399944,Fast Food Restaurant
300,Church and Wellesley,43.665860,-79.383160,A&W,43.666415,-79.378235,Fast Food Restaurant
303,Church and Wellesley,43.665860,-79.383160,Noah's Natural Foods,43.668532,-79.385885,Food & Drink Shop
352,Regent Park / Harbourfront,43.654260,-79.360636,The Healthy Road,43.656265,-79.357119,Health Food Store
395,"Garden District, Ryerson",43.657162,-79.378937,Crepe Delicious,43.654536,-79.380889,Fast Food Restaurant


In [35]:
toronto_food_venues.groupby('Venue').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude,Venue Category
Venue,,,,,,
A&W,3,3,3,3,3,3
Assembly Chef's Hall,3,3,3,3,3,3
Billy Bishop Café,1,1,1,1,1,1
Bourbon St. Grill,1,1,1,1,1,1
Chick-n-Joy,1,1,1,1,1,1
Crepe Delicious,1,1,1,1,1,1
FILM CAFE,1,1,1,1,1,1
FLOCK Rotisserie + Greens,1,1,1,1,1,1
Hogtown Smoke,3,3,3,3,3,3


In [36]:
toronto_food_venues.drop(['Neighborhood Latitude'],axis=1,inplace=True)
toronto_food_venues.drop(['Venue Latitude'],axis=1,inplace=True)
toronto_food_venues.drop(['Neighborhood Longitude'],axis=1,inplace=True)
toronto_food_venues.drop(['Venue Longitude'],axis=1,inplace=True)

/Users/rhiannon.smith/.pyenv/versions/3.7.5/envs/pm_monitoring/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [37]:
toronto_food_venues.drop(['Venue Category'],axis=1,inplace=True)

In [38]:
toronto_food_venues

,Neighborhood,Venue
1,The Beaches,The Big Carrot Natural Food Market
49,India Bazaar / The Beaches West,The Burger's Priest
62,India Bazaar / The Beaches West,KFC
80,Studio District,eastside social
109,Davisville North,Summerhill Market North
126,North Toronto West,A&W
300,Church and Wellesley,A&W
303,Church and Wellesley,Noah's Natural Foods
352,Regent Park / Harbourfront,The Healthy Road
395,"Garden District, Ryerson",Crepe Delicious


In [39]:
toronto_food_venues.groupby('Venue').count()

,Neighborhood
Venue,
A&W,3
Assembly Chef's Hall,3
Billy Bishop Café,1
Bourbon St. Grill,1
Chick-n-Joy,1
Crepe Delicious,1
FILM CAFE,1
FLOCK Rotisserie + Greens,1
Hogtown Smoke,3


In [40]:
toronto_food_venues.groupby('Neighborhood').count()

,Venue
Neighborhood,
Berczy Park,1
Business reply mail Processing CentrE,1
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,1
Church and Wellesley,2
Commerce Court / Victoria Hotel,2
Davisville North,1
First Canadian Place / Underground city,2
"Garden District, Ryerson",2
High Park / The Junction South,1


In [42]:
toronto_onehot = pd.get_dummies(toronto_food_venues[['Venue']], prefix="", prefix_sep="")
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_food_venues['Neighborhood'] )
toronto_onehot.shape

(32, 22)

In [44]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,A&W,Assembly Chef's Hall,Billy Bishop Café,Bourbon St. Grill,Chick-n-Joy,Crepe Delicious,FILM CAFE,FLOCK Rotisserie + Greens,Hogtown Smoke,...,Market 707,Noah's Natural Foods,Summerhill Market North,The Big Carrot Natural Food Market,The Burger's Priest,The Good Fork,The Healthy Road,West End Mamas,eastside social,iQ Food Co
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Business reply mail Processing CentrE,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,CN Tower / King and Spadina / Railway Lands / ...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Church and Wellesley,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Commerce Court / Victoria Hotel,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
